In [1]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)
    
from src.predictionModule.LoadupSamples import LoadupSamples
from src.predictionModule.MachineModels import MachineModels 

import numpy as np
import datetime
import pandas as pd
import polars as pl
import optuna
import scipy

import logging
formatted_date = datetime.datetime.now().strftime("%d%b%y_%H%M").lower()

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(fmt="%(asctime)s - %(message)s")
handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(handler)
else:
    logger.handlers[:] = [handler]

#Output File handler
formatted_str = f"notebook-lstm-optuna-{formatted_date}"
file_handler = logging.FileHandler(f"{formatted_str}.log", mode="w")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Usage
logger.info("This will print to the notebook's output cell")

2025-08-26 13:36:34,812 - This will print to the notebook's output cell


c:\Users\kimer\Desktop\RandomOdyssey\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import random
import numpy as np
import torch
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


In [3]:
params_default = {
    "idxAfterPrediction": 5,
    'timesteps': 90,
    'target_option': 'last',
    "LoadupSamples_time_scaling_stretch": True,
    "LoadupSamples_time_inc_factor": 10,
    
    "LSTM_units": 32,
    "LSTM_num_layers": 3,
    "LSTM_dropout": 0.001,
    "LSTM_recurrent_dropout": 0.001,
    "LSTM_learning_rate": 0.001,
    "LSTM_optimizer": "adam",
    "LSTM_bidirectional": True,
    "LSTM_batch_size": 2**12,
    "LSTM_epochs": 10,
    "LSTM_l1": 0.001,
    "LSTM_l2": 0.001,
    "LSTM_inter_dropout": 0.001,
    "LSTM_input_gaussian_noise": 0.001,
    "LSTM_conv1d": True,
    "LSTM_conv1d_kernel_size": 3,
    "LSTM_loss": "mse",
    "LSTM_transformer_before": False,
    "LSTM_transformer_after": False,
    "LSTM_tf_d_model": 0,
    "LSTM_tf_nhead": 4,
    "LSTM_tf_num_layers_before": 1,
    "LSTM_tf_num_layers_after": 1,
    "LSTM_tf_dim_feedforward": 128,
    "LSTM_tf_dropout": 0.1,
    "LSTM_tf_positional_encoding": True,
    "LSTM_tf_pool": "last",
}


In [4]:
stock_group = "group_regOHLCV_over5years"
eval_date = datetime.date(year=2025, month=6, day=13)
split_date = datetime.date(year=2023, month=12, day=31)

studytime = 60*60*4
studyname = f"sandbox_lstm_optuna_{formatted_str}"
n_startup_trials = 15

In [5]:
def objective(trial: optuna.Trial) -> float:
    q_full = 0.99
    opt_params = params_default.copy()
    opt_params["year_start"] = trial.suggest_int("year_start", 2020, 2022)
    opt_params["idxAfterPrediction"] = 4
    opt_params["LoadupSamples_time_inc_factor"] = trial.suggest_int("LoadupSamples_time_inc_factor", 61, 81, step=5)
    opt_params["timesteps"] = 75
    opt_params["LSTM_units"] = trial.suggest_categorical("LSTM_units", [16, 32])
    opt_params["LSTM_num_layers"] = 1
    opt_params["LSTM_learning_rate"] = trial.suggest_float("LSTM_learning_rate", 1e-4, 5e-4, log=True)
    opt_params["LSTM_epochs"] = 2
    opt_params["LSTM_l1"] = trial.suggest_float("LSTM_l1", 1e-5, 1e-4, log=True)
    opt_params["LSTM_l2"] = trial.suggest_float("LSTM_l2", 1e-4, 1e-2, log=True)
    opt_params["LSTM_dropout"] = trial.suggest_float("LSTM_dropout", 1e-3, 1e-2, log=True)
    opt_params["LSTM_inter_dropout"] = trial.suggest_float("LSTM_inter_dropout", 1e-2, 1e-1, log=True)
    opt_params["LSTM_recurrent_dropout"] = trial.suggest_float("LSTM_recurrent_dropout", 1e-3, 1e-2, log=True)
    opt_params["LSTM_conv1d_kernel_size"] = 3
    opt_params["LSTM_transformer_before"] = False
    opt_params["LSTM_transformer_after"] = False
    opt_params["LSTM_tf_nhead"] = 2
    opt_params["LSTM_tf_num_layers_before"] = 0
    opt_params["LSTM_tf_num_layers_after"] = 0
    opt_params["LSTM_tf_dim_feedforward"] = 8
    opt_params["LSTM_tf_dropout"] = 0.1
    opt_params["is_single_feature"] = False

    ls = LoadupSamples(
        train_start_date=datetime.date(year=opt_params["year_start"], month=1, day=1),
        test_dates=[eval_date],
        treegroup=None,
        timegroup=stock_group,
        params=opt_params,
    )
    ls.load_samples(main_path = "../src/featureAlchemy/bin/")

    ls.split_dataset(
        start_date=datetime.date(year=opt_params["year_start"], month=1, day=1),
        last_train_date=split_date,
        last_test_date=eval_date,
    )

    Xtrain = ls.train_Xtime
    ytrain = ls.train_ytime
    Xtest = ls.test_Xtime
    ytest = ls.test_ytime

    true_res = ls.meta_pl_test

    Xtrain = Xtrain[:, -opt_params["timesteps"]:, :]
    Xtest = Xtest[:, -opt_params["timesteps"]:, :]

    if opt_params["is_single_feature"]:
        Xtrain = Xtrain[:, :, [0]]
        Xtest = Xtest[:, :, [0]]

    mm = MachineModels(opt_params)

    starttime0 = datetime.datetime.now()
    model_lstm0, res_dict0 = mm.run_LSTM_transformer_torch(Xtrain, ytrain, Xtest, ytest, device="cuda")
    #preds_train0 = mm.predict_LSTM_transformer_torch(model_lstm0, Xtrain, batch_size=opt_params["LSTM_batch_size"], device="cuda")
    preds_test0 = mm.predict_LSTM_transformer_torch(model_lstm0, Xtest, batch_size=opt_params["LSTM_batch_size"], device="cuda")
    endtime0 = datetime.datetime.now()

    # final mask
    q2 = q_full
    mask_pred2_test_above = (preds_test0 >= np.quantile(preds_test0, q2))
    mask_pred2_test_below = (preds_test0 <= np.quantile(preds_test0, 1-q2))

    true_res_masked_above = true_res.filter(pl.Series(mask_pred2_test_above))
    true_res_masked_below = true_res.filter(pl.Series(mask_pred2_test_below))

    score = (scipy.stats.gmean(true_res_masked_above['target_ratio'].to_numpy())) ** (1/opt_params["idxAfterPrediction"])

    # Log some results
    def quant_dis_in_mask(mask: np.ndarray, q: float) -> float:
        if not mask.any():
            return len(mask)
        return np.quantile(np.abs(np.diff(np.where(mask)[0])), q)

    fullmask_above = mask_pred2_test_above.copy()
    fullmask_below = mask_pred2_test_below.copy()

    logger.info(f"Trial {trial.number} with params: {opt_params}")
    logger.info(f"  q2: {q2:.4f}")
    logger.info(f"  Duration0: {endtime0 - starttime0}")
    logger.info(f"  Val RMSE0 adjusted: {res_dict0['val_rmse']/opt_params['LoadupSamples_time_inc_factor']:.4f}")
    logger.info(f"  Mean all prediction: {scipy.stats.gmean(true_res['target_ratio'].to_numpy()):.4f}")
    logger.info(f"  Mean above prediction: {scipy.stats.gmean(true_res_masked_above['target_ratio'].to_numpy()):.4f}")
    logger.info(f"  Mean below prediction: {scipy.stats.gmean(true_res_masked_below['target_ratio'].to_numpy()):.4f}")
    logger.info(f"  Quantile 0.99 for distance in mask above: {quant_dis_in_mask(fullmask_above, 0.99)}")
    logger.info(f"  Quantile 0.99 for distance in mask below: {quant_dis_in_mask(fullmask_below, 0.99)}")
    logger.info(f"  Ratio for quantile-distance-to-length above: {quant_dis_in_mask(fullmask_above, 0.99) / len(fullmask_above):.4f}")
    logger.info(f"  Ratio for quantile-distance-to-length below: {quant_dis_in_mask(fullmask_below, 0.99) / len(fullmask_below):.4f}")
    logger.info(f"  Score: {score:.4f}")

    return score

optuna.logging.enable_propagation()
sampler = optuna.samplers.TPESampler(n_startup_trials=n_startup_trials)
study = optuna.create_study(
    study_name = studyname,
    storage="sqlite:///sandbox_optuna.db",
    direction="maximize",
    load_if_exists=True,
    sampler=sampler,
)
study.optimize(objective, timeout=studytime)

logger.info(f"Best parameters: {study.best_params}")
logger.info(f"Best score: {study.best_value}")

df: pd.DataFrame = study.trials_dataframe()
logger.info("\nTrials DataFrame:")
logger.info(df.sort_values("value").to_string())

param_importances = optuna.importance.get_param_importances(study)
logger.info("Parameter Importances:")
for key, value in param_importances.items():
    logger.info(f"{key}: {value}")

[I 2025-08-26 13:36:36,451] A new study created in RDB with name: sandbox_lstm_optuna_notebook-lstm-optuna-26aug25_1336


2025-08-26 13:36:36,451 - A new study created in RDB with name: sandbox_lstm_optuna_notebook-lstm-optuna-26aug25_1336
2025-08-26 13:36:43,984 - Non-finite/too-large values in train y time: 48 samples.
2025-08-26 13:36:44,000 - Removing 48 samples from training data.


Epochs:   0%|          | 0/2 [00:00<?, ?it/s]

2025-08-26 13:37:22,501 - Epoch 1/2 — Train RMSE: 0.4737 — Validation RMSE: 0.4148


Epochs:  50%|█████     | 1/2 [00:23<00:23, 23.63s/it]

...

2025-08-26 17:36:41,063 - Epoch 2/2 — Train RMSE: 0.4100 — Validation RMSE: 0.4020


Epochs: 100%|██████████| 2/2 [00:42<00:00, 21.46s/it]


2025-08-26 17:36:45,307 - Trial 203 with params: {'idxAfterPrediction': 4, 'timesteps': 75, 'target_option': 'last', 'LoadupSamples_time_scaling_stretch': True, 'LoadupSamples_time_inc_factor': 61, 'LSTM_units': 16, 'LSTM_num_layers': 1, 'LSTM_dropout': 0.00568038750239644, 'LSTM_recurrent_dropout': 0.002016203152677215, 'LSTM_learning_rate': 0.0001968477660776833, 'LSTM_optimizer': 'adam', 'LSTM_bidirectional': True, 'LSTM_batch_size': 4096, 'LSTM_epochs': 2, 'LSTM_l1': 3.2864461343008076e-05, 'LSTM_l2': 0.0028720245345559376, 'LSTM_inter_dropout': 0.012055520573505172, 'LSTM_input_gaussian_noise': 0.001, 'LSTM_conv1d': True, 'LSTM_conv1d_kernel_size': 3, 'LSTM_loss': 'mse', 'LSTM_transformer_before': False, 'LSTM_transformer_after': False, 'LSTM_tf_d_model': 0, 'LSTM_tf_nhead': 2, 'LSTM_tf_num_layers_before': 0, 'LSTM_tf_num_layers_after': 0, 'LSTM_tf_dim_feedforward': 8, 'LSTM_tf_dropout': 0.1, 'LSTM_tf_positional_encoding': True, 'LSTM_tf_pool': 'last', 'year_start': 2021, 'is_sing

[I 2025-08-26 17:36:45,374] Trial 203 finished with value: 1.0010085527576613 and parameters: {'year_start': 2021, 'LoadupSamples_time_inc_factor': 61, 'LSTM_units': 16, 'LSTM_learning_rate': 0.0001968477660776833, 'LSTM_l1': 3.2864461343008076e-05, 'LSTM_l2': 0.0028720245345559376, 'LSTM_dropout': 0.00568038750239644, 'LSTM_inter_dropout': 0.012055520573505172, 'LSTM_recurrent_dropout': 0.002016203152677215}. Best is trial 38 with value: 1.0030939915675743.


2025-08-26 17:36:45,374 - Trial 203 finished with value: 1.0010085527576613 and parameters: {'year_start': 2021, 'LoadupSamples_time_inc_factor': 61, 'LSTM_units': 16, 'LSTM_learning_rate': 0.0001968477660776833, 'LSTM_l1': 3.2864461343008076e-05, 'LSTM_l2': 0.0028720245345559376, 'LSTM_dropout': 0.00568038750239644, 'LSTM_inter_dropout': 0.012055520573505172, 'LSTM_recurrent_dropout': 0.002016203152677215}. Best is trial 38 with value: 1.0030939915675743.
2025-08-26 17:36:45,378 - Best parameters: {'year_start': 2020, 'LoadupSamples_time_inc_factor': 61, 'LSTM_units': 32, 'LSTM_learning_rate': 0.00019528650543392196, 'LSTM_l1': 1.580485787466957e-05, 'LSTM_l2': 0.00012102445546530657, 'LSTM_dropout': 0.0060859622228199165, 'LSTM_inter_dropout': 0.02206797275475179, 'LSTM_recurrent_dropout': 0.0013414558850708327}
2025-08-26 17:36:45,378 - Best score: 1.0030939915675743
2025-08-26 17:36:45,404 - 
Trials DataFrame:
2025-08-26 17:36:45,427 -      number     value             datetime_sta

In [6]:
df.to_parquet(f"sandbox_lstm_optuna_{formatted_str}.parquet", index=False)